#### initialize AI Platform 

```python
aiplatform.init(project=PROJECT_ID, location=REGION)
```

# Datasets

```python

src_uris = [IMPORT_FILE]
display_name = dataset_display_name

## check if dataset exists 
datasets = aiplatform.Datasets.list(filter = f"labels.series={model_display_name}")

# Create a new one if it doesn't
if datasets:
    dataset = datasets[0]
    print(f"Endpoint Exists: {datasets[0].resource_name}")
else:
    dataset = aiplatform.ImageDataset.create(
    display_name=display_name,
    labels = {'series' : f"{model_display_name}"}  
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.bounding_box,
    )
    print(f"Dataset Created: {dataset.resource_name}")
    
print(f'Review the Dataset in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{dataset.name}?project={PROJECT_ID}')

```

# Models

```python

modelmatch = aiplatform.Model.list(filter = f'display_name={model_display_name}_{EXPERIMENT} AND labels.series={model_display_name} AND labels.experiment={EXPERIMENT}')

#
# Create Model 
#

create_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        create_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if create_model:
    model = aiplatform.Model.create(
        display_name = f'{model_display_name}_{EXPERIMENT}',
        model_id = f'model_{model_display_name}_{EXPERIMENT}',
        parent_model =  modelmatch[0].resource_name,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{model_display_name}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )


#
# Upload Model 
#

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{model_display_name}_{EXPERIMENT}',
        model_id = f'model_{model_display_name}_{EXPERIMENT}',
        parent_model =  modelmatch[0].resource_name,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{model_display_name}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )
```

# Endpoints

```python

### Create/Retrieve The Endpoint For This Series

endpoints = aiplatform.Endpoint.list(filter = f"labels.series={model_display_name}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{model_display_name}",
        labels = {'series' : f"{model_display_name}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

```

# Jobs

```python
job_display_name = job_display_name

job = aiplatform.AutoMLImageTrainingJob(
    display_name=job_display_name,
    prediction_type="object_detection",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)
```

print(job)

# Experiments

```python
## Initialize Experiment 
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

## Get a List of all your experiments
experiments = aiplatform.Experiment.list()

## Check if the Experiment exists 
experiment = aiplatform.Experiment.list(filter=f"labels.series={experiment_name}")

if experiment:
    experiment = experiment[0]
else: 
    experiment = aiplatform.Experiment.create(display_name = model_display_name, labels = {'series' : f'{model_display_name}'})

```

# TensorBoard
```python

tb = aiplatform.Tensorboard.list(filter=f"labels.series={model_display_name}")

if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = model_display_name, labels = {'series' : f'{model_display_name}'})

```